## Necessary Packages

In [75]:
import argparse
import re
import numpy as np
import cairo

## Core functions

In [60]:
def parse_header(line):
    '''Parses fasta file header to obtain read info'''
    header = line.split(' ')
    gene = header[0]
    gene = gene[1:]
    loc = header[1].split(':')
    if len(header) == 4:
        stat = header[2] + header[3] # (reverse complement)
    else:
        stat = ''
    chrm = loc[0]
    pos = loc[1].split('-')
    base = int(pos[0])
    end = int(pos[1])
    return gene, chrm, base, end, stat

In [61]:
def build_exon(line, base, chr):
    '''This function is designed to build an exon dictionary entry
    from an uppercase segment in a fasta file.'''
    exon = ''
    start = base
    while line[chr].isupper():
        exon += line[chr]
        base += 1
        chr += 1
    fin = base - 1
    return base, start, exon, fin, chr

In [62]:
def build_intron(line, base, chr):
    '''This function is designed to build an intron dictionary entry
    from an uppercase segment in a fasta file.'''
    intron = ''
    start = base
    while chr < len(line) and line[chr].islower():
        intron += line[chr]
        base += 1
        chr += 1
    fin = base - 1
    intron = intron.upper()
    return base, start, intron, fin, chr

In [63]:
def parse_fa(fa_file):
    '''(file) -> dict, dict, dict
    This function is going to parse a fasta file where introns appear
    in lowercase and exons appear in uppercase. It will return an
    intron and an exon dictionary where the key represents the order
    of the segment.'''
    ln, base = 0, 0 # init counters
    new = True
    exons={} #key will be start coordinate, value will be seq, end coordinate
    introns={} #key will be start coordinate, value will be seq, end coordinate
    genes = {} #key will be gene abbr, value will be chr, start coor, end coor
    with open(fa_file) as fa:
        for line in fa:
            ln+=1
            line = line.strip('\n')
            if line.startswith('>'): # header
                if new == False:
                    new = True
                    char = 0
                    while char < len(prev_line):
                        if prev_line[char].isupper():
                            base, start, exon, fin, char = build_exon(prev_line, base, char)
                            exons[start] = [exon, fin]
                        elif prev_line[char].islower():
                            base, start, intron, fin, char = build_intron(prev_line, base, char)
                            introns[start] = [intron, fin]
                gene, chrm, base, end, stat = parse_header(line)
                if gene not in genes:
                    genes[gene] = [chrm, base, end]
                prev_line = ''
            else: # sequence line
                prev_line += line
                new = False
    char = 0
    while char < len(prev_line):
        if prev_line[char].isupper():
            base, start, exon, fin, char = build_exon(prev_line, base, char)
            exons[start] = [exon, fin]
        elif prev_line[char].islower():
            base, start, intron, fin, char = build_intron(prev_line, base, char)
            introns[start] = [intron, fin]
    return introns, exons, genes

In [64]:
def make_motif_pattern(motif):
    '''(string) -> string
    This function reads in a motif and builds a regex expression that will
    recognize all equivalent motifs.'''
    motif_pattern = ''
    trans_dict = {
        "A": "A",
        "C": "C",
        "G": "G",
        "T": "T",
        "U": "T",
        "M": "[AC]",
        "R": "[AG]",
        "W": "[AT]",
        "S": "[CG]",
        "Y": "[CT]",
        "K": "[GT]",
        "V": "[ACG]",
        "H": "[ACT]",
        "D": "[AGT]",
        "B": "[CGT]",
        "N": "[GATC]"}
    for char in motif:
        motif_pattern += trans_dict[char]
    return motif_pattern

In [78]:
def id_motif(m_file, introns, exons):
    '''(file, dict, dict) -> dict
    This function takes in a sequence as a string and uses regex to find motifs.
    Motifs are identified and returned with coordinates (start position?).'''
    motif_dict={}
    motif_coords = {} #dict for motif mapping
    # parse motif file
    with open(m_file) as motifs:
        for motif in motifs:
            motif = motif.strip('\n')
            motif_dict[motif] = make_motif_pattern(motif.upper())
    for motif in motif_dict:
        pattern = re.compile(motif_dict[motif])
        if motif not in motif_coords:
                motif_coords[motif]= []
        for intron in introns:
            coords = []
            for match in re.finditer(motif_dict[motif], introns[intron][0]):
                real_c = tuple(np.add(match.span(), (intron,intron)))
                coords.append(real_c) # adjusted coordinates of each match as np array
            for coord in coords:
                motif_coords[motif].append(coord)
        for exon in exons:
            coords = []
            for match in re.finditer(motif_dict[motif], exons[exon][0]):
                real_c = tuple(np.add(match.span(), (intron,intron)))
                coords.append(real_c) #coordinates of each match in tuple form
            for coord in coords:
                motif_coords[motif].append(coord)
    return motif_coords

## using pycairo to draw motifs

In [66]:
def id_exons(exon_dict, gene_info):
    '''(dict, list) -> list
    This function is meant to be used by draw_motifs() to id exons for drawing.
    exon_dict (key = true start pos, value = sequence, end pos)
    gene_info [chromosome (str), start pos (int), end pos (int)]'''
    exons = []
    start = gene_info[1]
    end = gene_info[2]
    chrom = gene_info[0]
    for entry in exon_dict:
        if entry > start and entry < end:
            exons.append((entry,exon_dict[entry][1])) #append tuple of exon coordinates to list
    return exons

In [104]:
def pick_motifs(m_list, gene_info):
    '''This function is meant to be used by draw_motifs() to id motifs
    for drawing on a particular gene.
    m_list (list of motif coordinates in tuple form)
    gene_info [chromosome (str), start pos (int), end pos (int)]'''
    motifs = []
    start = gene_info[1]
    end = gene_info[2]
    chrom = gene_info[2]
    for coordinates in m_list:
        if coordinates[0] > start and coordinates[0] < end:
            motifs.append(coordinates)
    return motifs

In [118]:
def draw_motifs(m_dict, i_dict, e_dict, g_dict):
    '''(dict,dict,dict,dict) -> svg
    This function uses dictionaries generated from parse_fa() and id_motif. Dictionaries 
    are: m_dict (motif dictionary, key = motif, value = list of start positions),
    i_dict (intron dictionary, key = true start pos, value = sequence, end pos),
    e_dict (exon dictionary, key = true start pos, value = sequence, end pos),
    g_dict (gene dictionary, key = gene name, value = chromosome, start pos, end pos).
    Function generates an SVG image of the gene including introns,
    exons and motif mapping, using pycairo to draw.'''
    g, m = 0, 0 # init gene & motif counter
    w = 1500 # set pixel width
    h = 1000 * len(g_dict) # pixel height = frame x each gene to draw
    colors = [[0.9,0.1,0.1], #red
              [0.4, 0.9, 0.4], #green
              [0.2, 0.23, 0.9], #blue
              [0.7, 0.7, 0.2], #yellow
              [0.2,0.7,0.5], #teal
              [0.57, 0.2, 0.7], #purple
              [0.7, 0.45, 0.2], #orange
              [0.1, 0.9, 0.1], #vv green
              [0.2, 0.7, 0.7] #light blue
             ]    # Up to 9 motif colors available, will repeat colors after 9
    for motif in m_dict:
        m_dict[motif].append(colors[m%9])
        m+=1
    with cairo.SVGSurface('motif_map.svg', w, h) as surface:
        context = cairo.Context(surface)
        for gene in g_dict:
            scale = 1300 / (g_dict[gene][2] - g_dict[gene][1]) #scale gene drawing to fit frame
            geneinfo = gene+': '+g_dict[gene][0]+': '+str(g_dict[gene][1])+'-'+str(g_dict[gene][2])
            #draw gene outline
            context.set_source_rgb(0,0,0) #drawing a black line
            context.set_line_width(10)
            context.move_to(100, 500 + 1000*g) # start position
            context.line_to(1400, 500 + 1000*g) # draw line to end pos
            context.stroke()
            #draw exons
            exon_list = id_exons(e_dict, g_dict[gene])
            for exon in exon_list:
                start = exon[0] - g_dict[gene][1]
                end = exon[1] - exon[0]
                context.rectangle((100+start*scale), (450+1000*g), ((end)*scale), 100)
                context.fill()
            #draw motifs
            for motif in m_dict:
                context.set_source_rgb(m_dict[motif][-1][0],m_dict[motif][-1][1],m_dict[motif][-1][2])
                motif_list = pick_motifs(m_dict[motif][:-1], g_dict[gene])
                for site in motif_list:
                    start = site[0] - g_dict[gene][1]
                    end = site[1] - site[0]
                    context.rectangle((100+start*scale), (450+1000*g), (end*scale), 100)
                    context.fill()
            #label genes
            context.set_source_rgb(0,0,0)
            context.move_to(100, (100+1000*g))
            context.set_font_size(40)
            context.show_text(geneinfo)
            #final things
            g += 1 # update gene count
        # make legend
        context.move_to(1200, 100)
        context.show_text('Motif Legend')
        context.set_font_size(20)
    return None

## just testing

In [68]:
motifs = ['ygcy','GCAUG','catag','YYYYYYYYYY']
introns = {121444593: ['TCTGCCTTTTGGGTAACTCTTTAGTATTTTAGCTTCTAGTTCCTCCTCTCTGCCCTGTTCTGCTGGTACTTTCTTCCAAGAAAAAGGAACTCTTAAGAGGATGAGAGTTGGCTAAAACAAAAAATAATCTTTCTTTCAACTGTGGCACTACCCTGGGTTGCCCGTCTGTGTTTTATTTTCCAAAGGCAAAAAAACCATTATGCTATGTTAAGCTAATTTCAATACAAGGCACACTTACTTGCTTCAGACTCTGCTTTGCTGTTGAACTTGTTTCCCTTTTAATTTTGTTTTAAATTGTACTTTTGCTTTTTTAGTAGCTAATAACATTCGGCTTTTTAATTTTTTGAATGTTACTAACTGTTCCCTGTTATCCCTTCCCTTGTTTTTTAATTTGCTTAAAG', 121444993], 
           121445018: ['GTAAGCGGTTTGTTTCTGTGATTGAGGTGTGTCTCACTTTCTCTGGTCCTCTGTTCTCACCGAGTGTTGCCCAGTACCCACTTGTCCTCACCAAGGTCCCTGCATGAGATGTTGCACGCTGTTTCTCCTCCTCTCTTTTCATTCTGTCGCCATTGGGTTTTTGCCCTACCTCTACCTGAGTTCTCCTTCCCTTTCTTGTTCAGTCTTTGTTCTTGTGAATTTTAGCACTAAAGTATGCTGCGTGTGGTTTTTATTGCTGAGTCCACACTCTGGGGAGGAGCATTGATGTCACACTGTGTGCTACTGTCCTCATGGACTGCCTGTCTTGGGTTTTATTACTCCTGAA', 121445363]}
exons = {7150537: ['AAAAACCTCTTCAGGCACTGGTGCCGAGGACCCTAG', 7150572], 
         152446704: ['ACTCAGTCGGCTGTCAAATCACTGAAGCGACCCCTCGAGGCAACCTTTGACCTG', 152446757], 
         28903700: ['GATAACTGTTCCCCCTCCTCCATCTCTGAGCCCGTGTCACAGGTATCACCCCCTTCTTGCCCTCAGCCCAGCTGCTGTGCCCCTGCCACCCGCGCCCCCTCAGCCCCTTGCGCGTCGCATCCAAGGT', 28903826], 
         121444994: ['ATGGTACTACCACTAAAGCGGAGG', 121445017]}
genes = {'INSR': ['chr19', 7150261, 7150808], 'MBNL': ['chr3', 152446461, 152447003], 
         'ATP2A1': ['chr16', 28903467, 28904044], 'CLASP1': ['chr2', 121444593, 121445363]}

In [69]:
motif_dict = {}
for motif in motifs:
    motif_dict[motif] = make_motif_pattern(motif.upper())
print(motif_dict)

{'ygcy': '[CT]GC[CT]', 'GCAUG': 'GCATG', 'catag': 'CATAG', 'YYYYYYYYYY': '[CT][CT][CT][CT][CT][CT][CT][CT][CT][CT]'}


In [98]:
motif_coords = {'ygcy': [(7150350, 7150354), (7150367, 7150371), (7150437, 7150441), (7150599, 7150603), (7150603, 7150607), (7150626, 7150630), (7150637, 7150641), (7150690, 7150694), (7150704, 7150708), (7150728, 7150732), (7150791, 7150795), (152446509, 152446513), (152446524, 152446528), (152446607, 152446611), (152446629, 152446633), (152446633, 152446637), (152446650, 152446654), (152446672, 152446676), (152446682, 152446686), (152446686, 152446690), (152446693, 152446697), (28903498, 28903502), (28903507, 28903511), (28903512, 28903516), (28903564, 28903568), (28903580, 28903584), (28903615, 28903619), (28903650, 28903654), (28903661, 28903665), (28903675, 28903679), (28903833, 28903837), (28903858, 28903862), (28903864, 28903868), (28903870, 28903874), (28903875, 28903879), (28903895, 28903899), (28903942, 28903946), (28903962, 28903966), (28903967, 28903971), (28903977, 28903981), (28903988, 28903992), (28904002, 28904006), (121444595, 121444599), (121444643, 121444647), (121444653, 121444657), (121444751, 121444755), (121444793, 121444797), (121444832, 121444836), (121444844, 121444848), (121444849, 121444853), (121444896, 121444900), (121444985, 121444989), (121445085, 121445089), (121445134, 121445138), (121445165, 121445169), (121445179, 121445183), (121445253, 121445257), (121445272, 121445276), (121445316, 121445320), (121445335, 121445339), (121445039, 121445043), (121445075, 121445079), (121445090, 121445094), (121445095, 121445099), (121445101, 121445105), (121445110, 121445114)], 
                'GCAUG': [(7150672, 7150677), (7150754, 7150759), (152446521, 152446526), (152446786, 152446791), (121445119, 121445124)], 
                'catag': [(7150799, 7150804), (152446933, 152446938), (28903695, 28903700)], 
                'YYYYYYYYYY': [(7150334, 7150344), (152446812, 152446822), (152446843, 152446853), (28903484, 28903494), (121444632, 121444642), (121444720, 121444730), (121444863, 121444873), (121444963, 121444973), (121445139, 121445149), (121445198, 121445208), (121445026, 121445036), (121445066, 121445076)]}

In [119]:
draw_motifs(motif_coords, introns, exons, genes)

In [38]:
len(genes)

4